In [59]:
import os, sys, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [2]:
dfoff = pd.read_csv('data/ccf_offline_stage1_train.csv')
dftest = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('data/ccf_online_stage1_train.csv')
print('data read end.')

data read end.


In [3]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [4]:
dftest.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


In [5]:
dfon.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513.0,NaN
1,13740231,34805,1,NaN,NaN,NaN,20160321.0
2,14336199,18907,0,NaN,NaN,NaN,20160618.0
3,14336199,18907,0,NaN,NaN,NaN,20160618.0
4,14336199,18907,0,NaN,NaN,NaN,20160618.0


In [10]:
#当数据集比较大的时候，必须加上null_counts参数，info才能显示非空数量
dfon.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11429826 entries, 0 to 11429825
Data columns (total 7 columns):
User_id          11429826 non-null int64
Merchant_id      11429826 non-null int64
Action           11429826 non-null int64
Coupon_id        872357 non-null object
Discount_rate    872357 non-null object
Date_received    872357 non-null float64
Date             10773928 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 610.4+ MB


In [17]:
dfoff.Discount_rate.isnull().sum()

701602

In [11]:
# 根据优惠券 `xx:yy` ，转换为折扣率 : `1 - yy/xx`，根据Discount_rate建立与折扣率相关的特征。
# getDiscountType 判断Discount_rate是否是 `xx:yy` 格式
# convertRate 转换xx:yy`格式优惠券格式为折扣率
# getDiscountMan 提取`xx:yy` 格式优惠卷的 `xx` 
# getDiscountJian 提取`xx:yy` 格式优惠卷的 `yy`

def getDiscountType(row):
    if pd.isnull(row):
        return np.nan
    elif ':' in str(row):
        return 1
    else:
        return 0

def convertRate(row):
    if pd.isnull(row):
        return 1.0
    elif ':' in str(row):
        rows = row.split(':')
        return 1.0 - float(rows[1]) /float(rows[0])
    else:
        return float(row)
    
def getDiscountMan(row):
    if ':' in str(row):
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0
    
def getDiscountJian(row):
    if ':' in str(row):
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

print("tool is ok.")

tool is ok.


处理数据

In [16]:
# 处理数据
def processData(df):
    # 转换 discunt_rate
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    print(df['discount_rate'].unique())
    # 转换 distance
    df['distance'] = df['Distance'].fillna(-1).astype(int)
    print(df['distance'].unique())
    return df

In [17]:
dfoff = processData(dfoff)

[1.         0.86666667 0.95       0.9        0.83333333 0.8
 0.5        0.85       0.75       0.66666667 0.93333333 0.7
 0.6        0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]
[ 0  1 -1  2 10  4  7  9  3  5  6  8]


In [19]:
dftest = processData(dftest)

[0.83333333 0.9        0.96666667 0.8        0.95       0.75
 0.98       0.5        0.86666667 0.6        0.66666667 0.7
 0.85       0.33333333 0.94       0.93333333 0.975      0.99      ]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


In [30]:
# 提取领取日期
date_received = dfoff['Date_received'].unique()
date_received = sorted(date_received[pd.notnull(date_received)])

In [32]:
# 提取使用日期
date_buy = sorted(dfoff[dfoff['Date'].notnull()]['Date'])

In [53]:
# 按领取日期分组计数
couponbydate = dfoff[dfoff['Date_received'].notnull()][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
couponbydate.columns = ['Date_received','count']

In [56]:
# 按使用日期分组计数
buybydate = dfoff[(dfoff['Date'].notnull()) & (dfoff['Date_received'].notnull())][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
buybydate.columns = ['Date_received','count']

In [57]:
def getWeekday(row):
    if row == 'nan':
        return np.nan
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

In [62]:
# 将领取日期转换为星期
dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

In [63]:
# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
dftest['weekday_type'] = dftest['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )

In [69]:
# 转换 weekday 为列
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
tmpdf = pd.get_dummies(dfoff['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

In [70]:
tmpdf = pd.get_dummies(dftest['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

In [71]:
def label(row):
    if pd.isnull(row['Date_received']):
        return -1
    if pd.notnull(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

In [72]:
# 判断优惠券是否在15天以内的使用的标签label
dfoff['label'] = dfoff.apply(label, axis = 1)

In [73]:
# 测试和验证数据
print("-----data split------")
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < 20160516)].copy()
valid = df[(df['Date_received'] >= 20160516) & (df['Date_received'] <= 20160615)].copy()
print("end")

-----data split------
end


In [77]:
df['Date_received'].describe()

count    1.053282e+06
mean     2.016032e+07
std      1.771416e+02
min      2.016010e+07
25%      2.016013e+07
50%      2.016032e+07
75%      2.016051e+07
max      2.016062e+07
Name: Date_received, dtype: float64

In [78]:
df['Date_received'].quantile(q=0.75)

20160514.0

In [74]:
# feature
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','distance', 'weekday', 'weekday_type'] + weekdaycols
print("----train-----")
model = SGDClassifier(#lambda:
    loss='log',
    penalty='elasticnet',
    fit_intercept=True,
    max_iter=100,
    shuffle=True,
    alpha = 0.01,
    l1_ratio = 0.01,
    n_jobs=1,
    class_weight=None
)
model.fit(train[original_feature], train['label'])

----train-----


SGDClassifier(alpha=0.01, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.01,
       learning_rate='optimal', loss='log', max_iter=100, n_iter=None,
       n_jobs=1, penalty='elasticnet', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [75]:
# #### 预测以及结果评价
print(model.score(valid[original_feature], valid['label']))

0.9094526220772331
